##### Copyright 2019 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hyperparameter Search

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/template/notebook.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/template/notebook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

### HyperParameter Tuning search
- `Hyperopt`: a popular Python library for optimizing over all sorts of complex
search spaces (including real values such as the learning rate, or discrete values
such as the number of layers).
- `Hyperas, kopt or Talos`: optimizing hyperparameters for Keras model (the first
two are based on Hyperopt).
- `Scikit-Optimize (skopt)`: a general-purpose optimization library. The Bayes
SearchCV class performs Bayesian optimization using an interface similar to Grid
SearchCV .
- `Spearmint`: a Bayesian optimization library.
- `Sklearn-Deap`: a hyperparameter optimization library based on evolutionary
algorithms, also with a GridSearchCV -like interface. [Link](https://github.com/rsteca/sklearn-deap)
- `keras-tuner`: 


## Setup

In [ ]:
! pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf
assert tf.__version__.startswith('2')

print(f'{tf.__version__}')

In [0]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = tf.keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(tf.keras.layers.Dense(1, **options))
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [ ]:
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
keras_reg.fit(X_train, Y_train, epochs=100, validation_data=(X_val, Y_val))

In [ ]:
from scipy.stats import reciprocal
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_srch = RandomizedSearchCV(keras_reg, param_distribs, cv=3, n_iter=10, verbose=2)
rnd_srch.fit(X_train, Y_train, validation_data=(X_val, Y_val))

## Notes

For general instructions on how to write docs for Tensorflow see [Writing TensorFlow Documentation](https://www.tensorflow.org/community/documentation).

The tips below are specific to notebooks for tensorflow.

### General

* Include the collapsed license at the top (this uses Colab's "Form" mode to hide the cells).
* Only include a single `H1` title.
* Include the button-bar immediately under the `H1`.
* Include an overview section before any code.
* Put all your installs and imports in a setup section.
* Always include the three `__future__` imports.
* Save the notebook with the Table of Contents open.
* Write python3 compatible code.
* Keep cells small (~max 20 lines).


### Working in GitHub

* Be consistent about how you save your notebooks, otherwise the JSON-diffs will be a mess.

* This notebook has the "Omit code cell output when saving this notebook" option set. GitHub refuses to diff notebooks with large diffs (inline images).

* [reviewnb.com](http://reviewnb.com) may help. You can access it using this bookmarklet:

  ```
javascript:(function(){ window.open(window.location.toString().replace(/github\.com/, 'app.reviewnb.com').replace(/files$/,"")); })()
 ```
 
* To open a GitHub notebook in Colab use the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) extension (or make a bookmarklet).
  
* The easiest way to edit a notebook in GitHub is to open it with Colab from the branch you want to edit. Then use File --> Save a copy in GitHub, which will save it back to the branch you opened it from.

* For PRs it's helpful to post a direct Colab link to the PR head: https://colab.research.google.com/github/{user}/{repo}/blob/{branch}/{path}.ipynb


### Code Style


* Notebooks are for people. Write code optimized for clarity.

* Demonstrate small parts before combining them into something more complex. Like below:

In [0]:
#Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(None, 5)),
    tf.keras.layers.Dense(3)
])

In [0]:
# Run the model on a single batch of data, and inspect the output.
result = model(tf.constant(np.random.randn(10,5), dtype = tf.float32)).numpy()

print("min:", result.min())
print("max:", result.max())
print("mean:", result.mean())
print("shape:", result.shape)

In [0]:
# Compile the model for training
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy)

* Keep examples quick. Use small datasets, or small slices of datasets. You don't need to train to convergence, train until it's obvious it's making progress.

* For a large example, don't try to fit all the code in the notebook. Add python files to tensorflow examples, and in the noptebook run: `!pip install git+https://github.com/tensorflow/examples`

### Code content

Use the highest level API that gets the job done (unless the goal is to demonstrate the low level API).

Use `keras.Sequential` > keras functional api > keras model subclassing > ...

Use  `model.fit` > `model.train_on_batch` > manual `GradientTapes`.

Use eager-style code.

Use `tensorflow_datasets` and `tf.data` where possible.

Avoid `compat.v1`.



### Text

* Use an imperative style. "Run a batch of images through the model."

* Use sentence case in titles/headings. 

* Use short titles/headings: "Download the data", "Build the Model", "Train the model".

